In [ ]:
## to run this code, you need to have the behavioral data dictionaries previously loaded in
## see 'loading_in_tar' and 'loading_in_npy' files in repo, limited code copied below:
import os
import numpy as np
import pickle

os.chdir('where/to/load')

vis_ctx = np.load("V1_spikes.npy", allow_pickle=True)
spikes_times = np.load('spikes_times.npy', allow_pickle=True)
goCue_times = np.load("goCue_times.npy", allow_pickle=True)
response_choices = np.load('response_choices.npy', allow_pickle=True)
feedback_types = np.load('feedback_types.npy', allow_pickle=True)
stim_times = np.load('stim_times.npy', allow_pickle=True)
vis_clusters = np.load('V1_clusters.npy', allow_pickle=True)
vis_timestamps = np.load('V1_timestamps.npy', allow_pickle=True)

## get trial types (contra/ipsi/nogo) for a given session
## calculate percent correct for each session

perc_corr = np.zeros(40)
sessions = np.arange(1, 40)
trial_type = {new_key: [] for new_key in sessions}

contra_clusters = []
contra_spiketimes = []
contra_starts = []
contra_ends = []
contra_beh = []

ipsi_clusters = []
ipsi_spiketimes = []
ipsi_starts = []
ipsi_ends = []
ipsi_beh = []

nogo_clusters = []
nogo_spiketimes = []
nogo_starts = []
nogo_ends = []

for s in sessions:
    print('session', s)
    trials = len(goCue_times[s]) # nTrials
    contra_Ctrials = np.zeros(trials) # tracking trials where correct response is contraleteral to probe
    contra_Itrials = np.zeros(trials) # tracking trials where INcorrect response is contraleteral to probe
    ipsi_Ctrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    ipsi_Itrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    nogo_trials = np.zeros(trials) # tracking NoGo trials
    
    for t in range(trials):
        
        response = int(response_choices[s][t]) # -1 right, 0 nogo, 1 left
        feedback = int(feedback_types[s][t]) # -1 incorrect, 1 correct
        lowerT = stim_times[s][t]
        upperT = goCue_times[s][t]
        
        this_trial_spikes = []
        this_cluster = []
        trial_start = []
        trial_end = []
        
        for key,value in vis_timestamps.items():
            if s == key:
                all_stamps = np.arange(1, len(vis_timestamps[s]))
                for i in all_stamps:
                    if lowerT <= vis_timestamps[s][i] < upperT: 
                        this_trial_spikes.append(float(vis_timestamps[s][i]))
                        this_cluster.append(int((vis_clusters[s][i])))
                        trial_start.append(lowerT)
                        trial_end.append(upperT)

        if response == 1: ## if this was a contralateral trial
            contra_spiketimes.extend(this_trial_spikes)
            contra_clusters.extend(this_cluster)
            contra_starts.extend(trial_start)
            contra_ends.extend(trial_end)
            if feedback == 1:
                contra_beh.append(1)
            elif feedback == -1:
                contra_beh.append(0)

        elif response == -1: ## if this was an ipsilateral trial
            ipsi_spiketimes.extend(this_trial_spikes)
            ipsi_clusters.extend(this_cluster)
            ipsi_starts.extend(trial_start)
            ipsi_ends.extend(trial_end)
            if feedback == 1:
                ipsi_beh.append(1)
            elif feedback == -1:
                ipsi_beh.append(0)

        elif response == 0: ## if this was a nogo trial
            nogo_spiketimes.extend(this_trial_spikes)
            nogo_clusters.extend(this_cluster)
            nogo_starts.extend(trial_start)
            nogo_ends.extend(trial_end)
            
contra_dict = {'n_neuron':contra_clusters, 'spiketimes':contra_spiketimes, 'starts': contra_starts, 'ends': contra_ends, 'behavior': contra_beh}
ipsi_dict = {'n_neuron':ipsi_clusters, 'spiketimes':ipsi_spiketimes, 'starts': ipsi_starts, 'ends': ipsi_ends, 'behavior': contra_beh}
nogo_dict = {'n_neuron':nogo_clusters, 'spiketimes':nogo_spiketimes, 'starts': nogo_starts, 'ends': nogo_ends}

os.chdir('where/to/save')

file_to_write = open("contra_dict.npy", "wb")
pickle.dump(contra_dict, file_to_write)

file_to_write = open("ipsi_dict.npy", "wb")
pickle.dump(ipsi_dict, file_to_write)

file_to_write = open("nogo_dict.npy", "wb")
pickle.dump(nogo_dict, file_to_write)